<a href="https://colab.research.google.com/github/Agamjot12/Sampling/blob/main/Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score

In [8]:
# Load your dataset
df = pd.read_csv('https://raw.githubusercontent.com/Agamjot12/Sampling/main/Creditcard_data.csv')

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = {'Time', 'Class'}, axis = 1), df['Class'], test_size=0.3, random_state=42)

In [10]:
y_train.value_counts()

0    534
1      6
Name: Class, dtype: int64

In [11]:
y_test.value_counts()

0    229
1      3
Name: Class, dtype: int64

In [14]:
# Define a dictionary of models to use
models = {
    'Logistic Regression': LogisticRegression(),
    'XGBoost': XGBClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [15]:
# Define a dictionary of sampling techniques to use
samplers = {
    'Random Over-Sampling': RandomOverSampler(),
    'Random Under-Sampling': RandomUnderSampler(),
    'SMOTE': SMOTE(),
    'Stratified Sampling': StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42),
    'Systematic Sampling': None,  # Replace None with the appropriate implementation
}

In [16]:
# Create an empty list to store the results
results = []

In [17]:
# Loop through each model and each sampling technique
for model_name, model in models.items():
    for sampler_name, sampler in samplers.items():
        # Apply the sampling technique to the training data
        if sampler_name == 'Stratified Sampling':
            # Stratified Sampling
            for train_idx, val_idx in sampler.split(X_train, y_train):
                X_resampled, y_resampled = X_train.iloc[train_idx], y_train.iloc[train_idx]
                X_val, y_val = X_train.iloc[val_idx], y_train.iloc[val_idx]

        elif sampler_name == 'Systematic Sampling':
            # Systematic Sampling
            step = 2
            indices = list(range(0, len(X_train), step))
            X_resampled, y_resampled = X_train.iloc[indices], y_train.iloc[indices]
        else:
            # All other resampling techniques
            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
        # Fit the model to the resampled data
        model.fit(X_resampled, y_resampled)
      
        # Make predictions on the test data
        y_pred = model.predict(X_test)
        
        # Compute the accuracy score of the model on the test data
        accuracy = accuracy_score(y_test, y_pred)
        # Append the results to the results list
        results.append({'Model': model_name, 'Sampler': sampler_name, 'Accuracy': accuracy})

In [18]:
# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

In [19]:
results_df

,Model,Sampler,Accuracy
0,Logistic Regression,Random Over-Sampling,0.935345
1,Logistic Regression,Random Under-Sampling,0.793103
2,Logistic Regression,SMOTE,0.926724
3,Logistic Regression,Stratified Sampling,0.982759
4,Logistic Regression,Systematic Sampling,0.982759
5,XGBoost,Random Over-Sampling,0.965517
6,XGBoost,Random Under-Sampling,0.797414
7,XGBoost,SMOTE,0.952586
8,XGBoost,Stratified Sampling,0.987069
9,XGBoost,Systematic Sampling,0.982759


In [20]:
# Reshape the DataFrame to have models as rows and samplers as columns
table = pd.pivot_table(results_df, values='Accuracy', index=['Model'], columns=['Sampler'])

In [21]:
table

Sampler,Random Over-Sampling,Random Under-Sampling,SMOTE,Stratified Sampling,Systematic Sampling
Model,,,,,
K-Nearest Neighbors,0.943966,0.668103,0.918103,0.987069,0.987069
Logistic Regression,0.935345,0.793103,0.926724,0.982759,0.982759
Random Forest,0.987069,0.612069,0.987069,0.987069,0.987069
Support Vector Machine,0.676724,0.405172,0.672414,0.987069,0.987069
XGBoost,0.965517,0.797414,0.952586,0.987069,0.982759
